In [1]:
# Sentiment Analysis

In [2]:
import pandas as pd

for big files when commiting to github https://stackoverflow.com/questions/32953238/how-can-i-ignore-big-files-and-push-to-git-repo


In [12]:

col_names = ['sentiment', 'id', 'date', 'query_string', 'user', 'text']
data_path = '/home/cate/streamlit-projects/data/sentiment140/training.1600000.processed.noemoticon.csv'

In [16]:
tweet_data = pd.read_csv(data_path, header = None, names = col_names, encoding="ISO-8859-1",engine = 'python' ).sample(frac = 1) #shaffles the data
tweet_data = tweet_data[['sentiment', 'text']] #disregard other columns
print(tweet_data.head())

sentiment                                               text
371032          0  @cali_hypocrisy i know!!  but u know u gotta d...
559896          0                              i cannot fall asleep 
26041           0  Finished 100 out of 131 questions for AP Euro....
821112          4  TweetDeckk &amp; iTunes Whattaa COMBOO  Love Ittt
589089          0  @jonacrombie dudee i knoww but i have to get u...


# Data Preprocessing

In [17]:
import re

allowed_chars = ' AaBbCcDdEeFfGgHhIiJjKkLlMmNnOoPpQqRrSsTtUuVvWwXxYyZz0123456789~`!@#$%^&*()-=_+[]{}|;:",./<>?'
punct = '!?,.@#'
maxlen = 280

def preprocess(text):
    return ''.join([' ' + char + ' ' if char in punct else char for char in [char for char in re.sub(r'http\S+', 'http', text, flags=re.MULTILINE) if char in allowed_chars]])[:maxlen]


Flair requires a specific format for the data to be in; it wants three CSV’s that look like:

\__label__ < LABEL>      < TEXT>

In [18]:
#apply the preprocessing function
tweet_data['text'] = tweet_data['text'].apply(preprocess)

In [19]:
# put __label__ in front of each sentiment
tweet_data['sentiment'] = '__label__' + tweet_data['sentiment'].astype(str)

In [24]:
#save the data
import os

# Create directory for saving data if it does not already exist
data_dir = '/home/cate/streamlit-projects/twitter_sentiment/processed-data/'
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

# Save a percentage of the data (you could also only load a fraction of the data instead)
amount = 0.125

tweet_data.iloc[0:int(len(tweet_data)*0.8*amount)].to_csv(data_dir + '/train.csv', sep='\t', index=False, header=False)
tweet_data.iloc[int(len(tweet_data)*0.8*amount):int(len(tweet_data)*0.9*amount)].to_csv(data_dir + '/test.csv', sep='\t', index=False, header=False)
tweet_data.iloc[int(len(tweet_data)*0.9*amount):int(len(tweet_data)*1.0*amount)].to_csv(data_dir + '/dev.csv', sep='\t', index=False, header=False)

# Classification

In this particular example we will use **Flair** This is a popular machine learning library for state-of-the-art NLP


Flair tutorials -> https://github.com/flairNLP/flair/tree/master/resources/docs

## Loading the data 
Loading the test, train and dev sets using Flair's NLPTaskDataFetcher class

In [28]:
from flair.datasets import ClassificationCorpus
from pathlib import Path

# load corpus containing training, test and dev data
corpus = ClassificationCorpus(Path(data_dir),
                                      test_file='test.csv',
                                      dev_file='dev.csv',
                                      train_file='train.csv')

2020-05-11 11:04:47,102 Reading data from /home/cate/streamlit-projects/twitter_sentiment/processed-data
2020-05-11 11:04:47,103 Train: /home/cate/streamlit-projects/twitter_sentiment/processed-data/train.csv
2020-05-11 11:04:47,104 Dev: /home/cate/streamlit-projects/twitter_sentiment/processed-data/dev.csv
2020-05-11 11:04:47,104 Test: /home/cate/streamlit-projects/twitter_sentiment/processed-data/test.csv


In [29]:
# a label dictionary to hold all the labels assigned to the text in the corpus

label_dict = corpus.make_label_dictionary()

2020-05-11 11:05:17,277 Computing label dictionary. Progress:
100%|██████████| 160000/160000 [00:34<00:00, 4653.80it/s]2020-05-11 11:05:52,824 [b'0', b'4']



In [31]:
# loading Flair's provided GloVe embeddings

from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings

word_embeddings = [WordEmbeddings('glove'),
# these are extra word embeddings provided by Flair for superior results
#                    FlairEmbeddings('news-forward'),
#                    FlairEmbeddings('news-backward')
                  ]

In [32]:
#initialize the embeddings

document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)

In [33]:
from flair.models import TextClassifier

classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

In [35]:
from flair.trainers import ModelTrainer

trainer = ModelTrainer(classifier, corpus)

In [ ]:
#train the model
trainer.train('model-saves',
              learning_rate=0.1,
              mini_batch_size=32,
              anneal_factor=0.5,
              patience=8,
              max_epochs=15)

# Testing out the model


In [ ]:
from flair.data import Sentence

classifier = TextClassifier.load('model-saves/final-model.pt')

pos_sentence = Sentence(preprocess('I love Python!'))
neg_sentence = Sentence(preprocess('Python is the worst!'))

classifier.predict(pos_sentence)
classifier.predict(neg_sentence)

print(pos_sentence.labels, neg_sentence.labels)